In [17]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import math
from skimage.feature import graycomatrix, graycoprops
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
import pandas as pd 

from processing import Processing
from texture import Texture
from laws import Laws
from histogramming import Histogramming
from visualizer_histogram import Visualizer_histogram
from GLCMAnalysis import GLCMAnalysis
from visualizer_glcm import Visualizer_glcm
from kmeans import Kmeans
from watershed_segmentation import Watershed_segmentation
from reg import Reg

data = []
image_names = [f'img/img_{i}.jpg' for i in range(1, 11)]
mask_names = [f'mask/mask_{i}.jpg' for i in range(1, 11)]
for i in range(0, 10):
    image_name = image_names[i]
    mask_name = mask_names[i]
    
    texture_processor = Texture(image_name)
    mean, variance, std_dev = texture_processor.get_texture()
    
    texture_processor1 = Texture(mask_name)
    mean1, variance1, std_dev1 = texture_processor1.get_texture()

    
    feature_vector = Laws.get_texture(image_name)
    
    feature_vector1 = Laws.get_texture(mask_name)

    
    glcm_analysis = GLCMAnalysis(image_name)
    glsm, features = glcm_analysis.get_texture()
    
    glcm_analysis1 = GLCMAnalysis(mask_name)
    glsm1, features1 = glcm_analysis1.get_texture()

    
    image = cv2.imread(image_name)
    mask = cv2.imread(mask_name)
    intersection = np.sum(np.minimum(image, mask))
    over = np.sum(np.maximum(image, mask))
    
    true_image = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)
    predicted_image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    true_image_bin = (true_image > 0).astype(np.uint8)
    predicted_image_bin = (predicted_image > 0).astype(np.uint8)
    true_positive = np.sum(true_image_bin & predicted_image_bin)
    false_positive = np.sum(~true_image_bin & predicted_image_bin)
    false_negative = np.sum(true_image_bin & ~predicted_image_bin)
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    l1_metric = np.mean(np.abs(true_image.astype(np.float32) - predicted_image.astype(np.float32)))

    
    w = Watershed_segmentation(image_name)
    water = w.get_segmentation()
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    intersection_w = np.sum(np.minimum(water, mask))
    over_w = np.sum(np.maximum(water, mask))

    true_image = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)
    true_image_bin = (true_image > 0).astype(np.uint8)
    predicted_image = water
    predicted_image_bin = (predicted_image > 0).astype(np.uint8)
    true_positive = np.sum(true_image_bin & predicted_image_bin)
    false_positive = np.sum(~true_image_bin & predicted_image_bin)
    false_negative = np.sum(true_image_bin & ~predicted_image_bin)
    precision_w = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall_w = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    l1_metric_w = np.mean(np.abs(true_image.astype(np.float32) - predicted_image.astype(np.float32)))
    
    k = Kmeans(image_name)
    km = k.get_segmentation()
    intersection_k = np.sum(np.minimum(km, mask))
    over_k = np.sum(np.maximum(km, mask))

    true_image = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)
    true_image_bin = (true_image > 0).astype(np.uint8)
    predicted_image = km
    predicted_image_bin = (predicted_image > 0).astype(np.uint8)
    true_positive = np.sum(true_image_bin & predicted_image_bin)
    false_positive = np.sum(~true_image_bin & predicted_image_bin)
    false_negative = np.sum(true_image_bin & ~predicted_image_bin)
    precision_k = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall_k = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    l1_metric_k = np.mean(np.abs(true_image.astype(np.float32) - predicted_image.astype(np.float32)))

    
    data.append({'Имя изображения': image_name, 
                 'Имя маски': mask_name, 
                 'Среднее значение изображения': mean, 
                 'Среднее значение маски': mean1, 
                 'Дисперсия изображения': variance, 
                 'Дисперсия маски': variance1, 
                 'Стандартное отклонение изображения': std_dev, 
                 'Стандартное отклонение маски': std_dev1, 
                 'Текстурные характеристики Laws изображения': feature_vector,
                 'Текстурные характеристики Laws маски': feature_vector1,
                 'Признаки изображения': features,
                 'Признаки маски': features1,
                 'Матрица взаимной встречаемости пикселей изображения': glsm,
                 'Матрица взаимной встречаемости пикселей маски': glsm1,
                 'intersection изображения и маски': intersection,
                 'over изображения и маски': over, 
                 'precision изображения и маски': precision,
                 'recall изображения и маски': recall,
                 'l1_metric изображения и маски': l1_metric,
                 'intersection watershed_segmentation и маски': intersection_w,
                 'over watershed_segmentation и маски': over_w,
                 'precision watershed_segmentation и маски': precision_w,
                 'recall watershed_segmentation и маски': recall_w,
                 'l1_metric watershed_segmentation и маски': l1_metric_w,
                 'intersection kmeans и маски': intersection_k,
                 'over kmeans и маски': over_k, 
                 'precision kmeans и маски': precision_k,
                 'recall kmeans и маски': recall_k,
                 'l1_metric kmeans и маски': l1_metric_k})
df = pd.DataFrame(data)
df.to_excel('results.xlsx', index=False)

In [18]:
df

,Имя изображения,Имя маски,Среднее значение изображения,Среднее значение маски,Дисперсия изображения,Дисперсия маски,Стандартное отклонение изображения,Стандартное отклонение маски,Текстурные характеристики Laws изображения,Текстурные характеристики Laws маски,...,intersection watershed_segmentation и маски,over watershed_segmentation и маски,precision watershed_segmentation и маски,recall watershed_segmentation и маски,l1_metric watershed_segmentation и маски,intersection kmeans и маски,over kmeans и маски,precision kmeans и маски,recall kmeans и маски,l1_metric kmeans и маски
0,img/img_1.jpg,mask/mask_1.jpg,127.972183,72.168625,221.171208,13071.662011,14.871826,114.331369,"[1164.2781, 380.09302, 234.45267, 325.19476, 2...","[9343.763, 1327.9404, 506.94266, 761.2168, 413...",...,2998362,23093092,0.379088,0.789348,306.621246,2368308,10725766,0.395065,1.0,127.524689
1,img/img_2.jpg,mask/mask_2.jpg,127.958893,62.046112,212.394947,11864.568644,14.573776,108.924601,"[1051.5374, 347.12003, 223.74545, 322.3448, 21...","[8034.6235, 1114.2964, 418.91138, 633.0674, 34...",...,2423874,18299476,0.326420,0.809375,242.242462,2035149,10391905,0.342117,1.0,127.513977
2,img/img_3.jpg,mask/mask_3.jpg,127.924530,79.065140,234.888347,13779.672988,15.326068,117.386852,"[1213.3213, 396.63818, 239.86227, 328.32312, 2...","[9720.414, 1362.591, 511.2531, 762.0821, 411.3...",...,3265313,22403039,0.412288,0.789461,292.018524,2595072,10948003,0.430862,1.0,127.455612
3,img/img_4.jpg,mask/mask_4.jpg,127.904053,68.831299,212.205272,12700.369825,14.567267,112.695918,"[1097.2693, 347.90308, 218.35805, 311.97256, 2...","[8727.166, 1184.7958, 442.53564, 692.69, 365.4...",...,2741260,19630878,0.352913,0.792905,257.715118,2255674,10617082,0.373337,1.0,127.584961
4,img/img_5.jpg,mask/mask_5.jpg,128.001114,82.967987,211.287673,14133.795057,14.535738,118.885639,"[1164.8923, 371.5231, 219.83504, 306.5974, 190...","[10247.749, 1423.2129, 530.7139, 805.04865, 42...",...,3392072,23947816,0.435940,0.789921,313.655762,2732837,11078422,0.454178,1.0,127.343521
5,img/img_6.jpg,mask/mask_6.jpg,127.939453,80.660370,230.588772,13928.984596,15.185150,118.021119,"[1179.1942, 377.07965, 234.17065, 331.38068, 2...","[9826.141, 1355.5682, 511.88483, 780.36084, 41...",...,3286133,22870446,0.418370,0.790347,298.832886,2638647,10990720,0.436615,1.0,127.442520
6,img/img_7.jpg,mask/mask_7.jpg,128.043350,83.028580,197.329923,14148.355217,14.047417,118.946859,"[1150.468, 361.0573, 210.1145, 295.6095, 177.1...","[9861.328, 1411.6312, 528.1931, 804.647, 427.6...",...,3423395,22603620,0.422662,0.788348,292.666992,2729817,11078154,0.442368,1.0,127.385513
7,img/img_8.jpg,mask/mask_8.jpg,127.941315,78.281433,221.678014,13700.630268,14.888855,117.049691,"[1176.296, 380.894, 232.14868, 321.46167, 208....","[9872.939, 1367.0862, 516.5635, 790.6244, 421....",...,3452930,24001280,0.413754,0.793249,313.542938,2563301,10915606,0.429016,1.0,127.446060
8,img/img_9.jpg,mask/mask_9.jpg,127.956497,77.702957,216.584091,13653.798316,14.716796,116.849469,"[1130.8826, 359.37152, 226.61935, 324.00204, 2...","[9419.066, 1287.9421, 487.71875, 775.69073, 40...",...,3318184,22495737,0.396561,0.791276,292.626221,2542634,10901681,0.416260,1.0,127.548935
9,img/img_10.jpg,mask/mask_10.jpg,128.020874,81.847977,238.592399,14037.606970,15.446436,118.480408,"[1247.1267, 402.68216, 241.95651, 338.39447, 2...","[9834.576, 1332.2932, 496.3617, 773.2469, 405....",...,3543909,23174978,0.430479,0.793459,299.546326,2692708,11041359,0.445114,1.0,127.390305
